# 1.爬取银行间线上资金

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re,datetime,json,time
from pymongo import MongoClient
import urllib


#lianjiez=''
chrome_options = Options()
# 无头模式启动
chrome_options.add_argument('--headless')
# 谷歌文档提到需要加上这个属性来规避bug
chrome_options.add_argument('--disable-gpu')
client=MongoClient('mongodb://root:' + '5768116' + '@121.196.220.14')





def gongkai():
    lianjiez=[]
    db = client.yinhangjian          #得到数据库
    collection = db.yinhangjian    
    
# 初始化实例
    driver= webdriver.Chrome(chrome_options=chrome_options)
# 打开主页
    es=driver.get("http://www.chinamoney.com.cn/chinese/mkdatapm/")
    time.sleep(5) 

    stime=driver.find_elements_by_xpath('.//div[@class="san-grid-f-r"]//span')

    date0=stime[0].text.split(' ')[0]
    time0=stime[0].text.split(' ')[1]

    d=driver.find_elements_by_xpath('.//ul[@class="san-tabs-a"]//a')

    zong=pd.DataFrame({ })
    for i in range(0,2):
        d[i].click()
        time.sleep(10) 
        t=driver.find_elements_by_xpath('.//table[@class="san-sheet-alternating"]//tr')
        print(len(t))
        for j in range(1,len(t)):
      

            
            
            if t[j].text!='' and '品种' not in t[j].text:
                zong1=pd.DataFrame({'品种': [t[j].text.split(' ')[0]],
                               '加权利率': [t[j].text.split(' ')[1]],
                                '最新利率': [t[j].text.split(' ')[2]],
                                '日期':[date0],
                                '时间':[time0],
                                '品种2':[i]
                         })
                zong=pd.concat([zong,zong1])
        zong= zong.reset_index(drop=True)    #重新定义索引

        print(zong)
        
        records = json.loads(zong.T.to_json()).values()
        collection.insert(records)
      
        zong=pd.DataFrame({ })


            



            

    print("Clean...")
    driver.close()
    driver.quit()
    
  
def run_function():  
    s = sched.scheduler(time.time, time.sleep)  
    s.enter(0, 2, event_func)  
    s.run()  


def timer1():  
    while True:  
        #sched模块不是循环的，一次调度被执行后就Over了，如果想再执行，可以使用while循环的方式不停的调用该方法  
        gongkai()
        time.sleep(900)  
        print('start')

#timer1()  
if __name__ == '__main__':
    timer1()


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: use options instead of chrome_options


33
        品种    加权利率    最新利率     日期     时间  品种2
0  DIBO001  2.5831  2.5800  08-30  16:30    0
1  DIBO007  2.7825  2.8500  08-30  16:30    0
2  DIBO014  2.7755  3.4000  08-30  16:30    0
3  DIBO021  2.6000  2.6000  08-30  16:30    0
4   DIBO1M  3.0150  2.9500  08-30  16:30    0
5   DIBO2M  2.9628  3.2000  08-30  16:30    0
6   DIBO3M  3.5000  3.5000  08-30  16:30    0
7   DIBO4M  3.6000  3.6000  08-30  16:30    0
8   DIBO6M  3.6042  3.5000  08-30  16:30    0


AutoReconnect: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。